In [104]:
import requests
import csv
import os
import pandas as pd
from bs4 import BeautifulSoup as bs
from urllib.parse import quote
import re
#from urllib.request import urlopen

In [105]:
#test2 #allot easier proably just going to use this 
#creates inflections for nouns and adjectives
#creates conjugations for verbs
def generate_variations(word):
        #test words
        #'Ῥωμαῖος' romans 
        #'μισώ' hate
        #'εἰμί' are to be
        #word = word.lower()

        #https://en.wiktionary.org/wiki/Ῥωμαῖος
        url= requests.get(f'https://en.wiktionary.org/wiki/{quote(word)}')
        soup = bs(url.text, 'html.parser')
        
        #remove unwanted classes
        for tag in soup.find_all('span',{'class':'tr Latn'}):
                tag.decompose()
        for tag in soup.find_all('table',{'class':'audiotable'}):
                tag.decompose()
        #HEADERS ---------------------------
        #find all tables 
        tables =soup.find_all('table')
        h3 =[]
        h4 =[]
        nav=[]
       
                
                        
        # for each table find the preceding header and sub header
        
        for i,item in enumerate(tables): 
                h3.append(tables[i].find_previous('h3').text.replace('[edit]','').lower())
                h4.append(tables[i].find_previous('h4').text.replace('[edit]','').lower())
                nav.append(tables[i].find_previous('div',{'class':'NavHead'}).text.replace('hide ▲','')
                .replace('/','-').replace(':','-').strip().lower())
                #  : / mess up the file path so you need to remove them
        #print(h3)
        #print(h4)
        #-------------------------
        
        #DATAFRAMES----------------------------------
        
        dataframes = pd.read_html(soup.prettify())
        for i,df in enumerate(dataframes):
                ##display(df)
                ##CREATE A FOLDER
                root = 'variations'
                os.makedirs(f'{root}/{h3[i]}/{word.lower()}/',exist_ok=True)
                print(f'{root}/{h3[i]}/{word}/')
                ##os.makedirs(nav[i],exist_ok=True)
                
                #print(h3[i])
                #print(h4[i])
                #print(nav[i])
                #print('WRITE TO FILE--------------------------------')
                
                df=dataframes[i]
                #display(df)
                df=df.dropna(how='all',axis=0)
                
                df=df.drop(list(df.filter(regex = 'Unnamed')), axis = 1)
                
                ##df =df.loc[:,~df.columns.str.startswith('Unnamed')]
                df=df.replace({'➤':''}, regex=True)
                df=df.replace({'•':''}, regex=True)
                
                df = df[df.iloc[:,0].str.contains("Notes") == False]
                #df= df.str.lower()
                ##df = df.style.set_properties(**{'text-align': 'middle'})
                df.to_csv(f'{root}/{h3[i]}/{word.lower()}/{nav[i]}_{h4[i]}.csv',index=False)
                
                
                #display(df)
                ## READ 
                #print('READ FROM FILE ------------------------------------')
                #display(pd.read_csv(f'{root}/{h3[i]}/{word}/{nav[i]}_{h4[i]}.csv'))
        #----------------------------------
generate_variations('μισώ')

        #test words
        #'Ῥωμαῖος' romans 
        #'μισώ' hate
        #'εἰμί' is/are to be
        #ελληνικά greeks




variations/verb/μισώ/


In [106]:
# THESE ARE THE MAIN TYPES OF SNETIMENT EXPRESSIONS THAT I CAN THINK OF 
#Accusitive(plural),opinion(singular) - I hate romans -         μισώ τους Ρωμαίους
#Nomitive(plural),opinion(plural)-      romans hate us -        οι Ρωμαίοι μας μισούν
#Nomitive,adjective -                   romans are evil -       οι Ρωμαίοι είναι κακοί
#Nomitive,simile-                       romans are like sheep - οι Ρωμαίοι είναι σαν πρόβατα
#Nomitive,metaphore-                    romans are snakes -     οι Ρωμαίοι είναι φίδια
#(NEGATION)Accusitive,opinion -         I dont hate romans -    δεν μισώ τους Ρωμαίους
#(NEGATION)Nomitive,adjective           romans are not evil -   οι Ρωμαίοι δεν είναι κακοί
#complex
#i like how well mannerd the greeks are  -postive
# i dislike the fact that romans are so nice  neutral

# will short scentences be enought for longer scentences like thes maybe idk
# we could do a semi procedural approach with templates and masking 
#we only come up with a few thousand sentment scentences then 
#TWO STAGE APSECT FIRST ner identify the aspects of the scentence 
#aspect expects aspects as input it does not identify the aspects we need both


def test():
    #PRESENT TEST VERB SINGULAR + ACCUSITIVE NOUN 
    
    #verb + 
    pass

#bad 
#hate
def accusitive_opinion_generator():
    #VERB + ACCUSITIVE NOUN
    pass
def nomitive_opinion_generator():
    #NOMITIVE NOUN + #VERB 
    #METAPHORES
    pass
def nomitive_simile_generator():
    pass

def nomitive_negation_generator():
    pass
def accusitive_negation_generator():
    pass

In [107]:

#go through each file ------
# for each file in a path do something
def to_folder(path):
    # converts non folders into folders
    root = f"{path}"
    if root[-1] != "/":
        root+="/"
    return root
    
def get_each_file_in(path):
    # takes in Noun, Adjective,Verb
    #print("test")
    filelist =[]
    root =to_folder(path)    
    for subdir, dirs, files in os.walk(root):
        for file in files:
            f = os.path.join(subdir, file)
            filelist.append(f)
    return filelist
            
            
# for each csv in a path do something         
def get_each_csv_in(path):
    # takes in Noun, Adjective,Verb
    #print("test")
    filelist =[]
    root =to_folder(path)    
    for subdir, dirs, files in os.walk(root):
        for file in files:
            f = os.path.join(subdir, file)
            if f.endswith('.csv'):
                filelist.append(f)
    return filelist
    
def get_each_df_in(path):
    # takes in Noun, Adjective,Verb
    #print("test")
    dflist =[]
    root =to_folder(path)    
    for f in get_each_csv_in(path):
        df = pd.read_csv(f)
        dflist.append(df)
    return dflist
    
def get_greek_words_in(path):
    wordlist = []
    for df in get_each_df_in(path):
        for i in df.index:
             wordlist.append(df.iloc[i,1])
    return wordlist
    
def init_variations_in(path):
    for word in get_greek_words_in(path):
        #display(df.iloc[i,1])
        generate_variations(word)

def match_words_in(searchpath,wordpath):
    filelist = []
    searchpath= to_folder(searchpath)
    for word in get_greek_words_in(wordpath):
        if(os.path.exists(searchpath + word)):
            filelist.append(get_each_file_in(searchpath + word))
    flat_list = [item for sublist in filelist for item in sublist]
    return flat_list
def path_to_df(filelist):
    dflist=[]
    for file in filelist:
        df = pd.read_csv(file)
        dflist.append(df)
    return dflist
        
def display_list(list):
    for i in list:
        display(i)


def get_accustive_nouns(dflist):
    dflist2=[]
    for df in dflist:
       filt= df.iloc[:,0].str.lower() == 'accusative'
       #dflist2.append(df[filt])
       dflist2.append(df[filt])
    return dflist2




def search_for_folder(folder):
    pass

   
def init_verbs(path):
    pass
    
def getrows(df):
    #should it return a pandas or array
    return df
    
def getcolumns():
    pass 




In [108]:
#get_each_file_in("dictionary/nouns")
#get_each_csv_in("dictionary/nouns")
#init_variations_in("dictionary/nouns")
matchedwords= match_words_in("variations/noun","dictionary/nouns")
display_list(get_accustive_nouns(path_to_df(matchedwords)))


,Case / #,Singular,Dual,Plural
3,Accusative,τὸν Ῥωμαῖον,τὼ Ῥωμαίω,τοὺς Ῥωμαίους


,case \ number,singular,plural
2,accusative,Ελληνίδα,Ελληνίδες


,case \ number,singular,plural
2,accusative,Αθηναίο,Αθηναίους


,Case / #,Singular,Dual,Plural
3,Accusative,τὴν Σπᾰρτῐᾱ́τῐδᾰ,τὼ Σπᾰρτῐᾱ́τῐδε,τᾱ̀ς Σπᾰρτῐᾱ́τῐδᾰς


,Case / #,Singular,Dual,Plural
3,Accusative,τὴν Τρωᾰ́δᾰ,τὼ Τρωᾰ́δε,τᾱ̀ς Τρωᾰ́δᾰς


...

,Case / #,Singular,Dual,Plural
3,Accusative,τὸν Δωρῐέᾰ,τὼ Δωρῐῆ,τοὺς Δωρῐέᾰς


,Case / #,Singular,Dual,Plural
3,Accusative,τὸν Δωρῐέᾱ / Δωρῐᾶ,τὼ Δωρῐῆ,τοὺς Δωρῐέᾱς / Δωρῐᾶς


,Case / #,Singular,Dual,Plural
3,Accusative,τὸν Ἴωνᾰ,τὼ Ἴωνε,τοὺς Ἴωνᾰς


,case \ number,singular,plural
2,accusative,Αλγερινό,Αλγερινούς


,case \ number,singular,plural
2,accusative,Αιθίοπα,Αιθίοπες


In [109]:
#SCRAPS
#part of speech 
# we could also do a hybrid approach use a ml for part of speech tagging then search the databasd for that in the opposite direction


'''
def get_matches(sentence,word):
    for match in re.finditer(word, sentence):
        print (match.start(), match.end())

def match_variations(partofspeech, sentence): 
    def checkformatch(word):
        get_matches(sentence,word)
        
    for_each_variation(partofspeech, checkformatch)
    
def for_each_variation(partofspeech,callback): 
    def for_every_cell(f):
        df = pd.read_csv(f)
        columns = df.columns.tolist()
        for _, i in df.iterrows():
            for c in columns:
                callback(i[c])
                #print(i[c])
                #print("############")
        
    for_each_file(partofspeech, for_every_cell)
    '''
    
    #SEARCH FOR ROW 
#PRING ROW AND COLUMNS
            
# outputs the start and end indexes of all occurences            

# ____________________________________________
        
    # input scentence is a string 
    #go into noun folder
    # for each item in each csv check if present in  
    #go into each word folder 
    # read each csv item
    # if it finds a match return index span of string row and column

def search_left():
    pass 
def search_right():
    pass 
def search_verbs():
    pass

def rule_based_sentiment():
    #SEARCH FOR NOUN RETURN SPAN
    
    
    #SERACH LEFT FOR VERB ACCUSITIVE 
    #POSSIBLE DETECT WHICH WAY TO SERACH BASED ON THE INFLECTION OF THE NOUN WHICH COLUMN IT BELONG TOO
    #POSSIBLY OPTIMIZE BY SEARCHING FOR SPECIFIC COLUMNS
    #HANDLE NEGATIONS 
    
    #SERCH RIGHT FOR INFINITIVE TO BE YOU ARE BAD
    #SERACH FOR NEGATIVE VERB OR NEGATIVE NOUN OR NEGATIVE ADJECTIVE 
    #HANDLE NEGATIONS 
    
    #EDGECASES FUNCTION
    #CAN AN ACCUSITIVE VERB SEARCH 
    
    #SEARCH DATIVE CASE  
    #I WOULD DIE FOR THE ROMANS positive
    #ROMANS SHOULD ALL DIE
    # I WOULD KILL ALL THE ROMANS IF I COULD
    # there is nothing worse than being a roman
    
    pass
    
    
    
#for_each_file("Noun",display)
#for_each_variation("Noun",display)
#get_matches("μισώ τους Ρωμαίους","τους Ρωμαίους")

#match_variations("Noun","μισώ τους Ρωμαίους")

#READ COULMN TWO OF VERB 
#READ COLUMN TWO OF NOUN 

# TEMPLATE
#μου αρέσουν οι Ρωμαίοι i like romans

#αγαπώ τους Ρωμαίους i love romans
#μισώ τους Ρωμαίους i hate romans
#περιφρονώ τους Ρωμαίους
#σιχαίνομαι τους Ρωμαίους

#σκοτώνω Ρωμαίους i kil romans

#Θέλω να σκοτώσω Ρωμαίους i want to kill romans
#Θέλω να πληγώσω τους Ρωμαίους i want to hurt romans





